In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('sales_train.csv')
items = pd.read_csv('items.csv')
item_cats = pd.read_csv('item_categories.csv')
shops = pd.read_csv('shops.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [3]:
train['date'] = pd.to_datetime(train['date'], dayfirst=True)

In [4]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   date            datetime64[ns]
 1   date_block_num  int64         
 2   shop_id         int64         
 3   item_id         int64         
 4   item_price      float64       
 5   item_cnt_day    float64       
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 134.4 MB


In [8]:
train['item_price'].describe()

count    2.935849e+06
mean     8.908532e+02
std      1.729800e+03
min     -1.000000e+00
25%      2.490000e+02
50%      3.990000e+02
75%      9.990000e+02
max      3.079800e+05
Name: item_price, dtype: float64

In [15]:
# removing records where item_price is negative
train = train[~(train['item_price']<0)]

# jan 2013

In [18]:
first_month = train[train['date_block_num']==0]

In [19]:
first_month['day_'] = first_month['date'].dt.day

In [20]:
first_month.groupby('day_').sum()['item_cnt_day']

day_
1     1951.0
2     8198.0
3     7422.0
4     6617.0
5     6346.0
6     5858.0
7     4984.0
8     4772.0
9     3162.0
10    3142.0
11    3829.0
12    5443.0
13    4475.0
14    2595.0
15    3137.0
16    3033.0
17    3096.0
18    3682.0
19    6044.0
20    5111.0
21    2636.0
22    2775.0
23    2727.0
24    3016.0
25    4244.0
26    6267.0
27    5273.0
28    2852.0
29    2764.0
30    3042.0
31    2986.0
Name: item_cnt_day, dtype: float64

# breaking train into every month

In [21]:
train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-01-02,0,59,22154,999.00,1.0
1,2013-01-03,0,25,2552,899.00,1.0
2,2013-01-05,0,25,2552,899.00,-1.0
3,2013-01-06,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.00,1.0
2935845,2015-10-09,33,25,7460,299.00,1.0
2935846,2015-10-14,33,25,7459,349.00,1.0
2935847,2015-10-22,33,25,7440,299.00,1.0


In [22]:
tot_month = train['date_block_num'].nunique()

names_=[]
for i in range(tot_month):
    names_.append('df_{}'.format(i))

In [23]:
for i in range(tot_month):
    names_[i] = train[train['date_block_num']==i]

# Group by for every month

In [24]:
df_0 = names_[0].drop(['date', 'item_price', 'date_block_num'],axis=1).groupby(['shop_id','item_id']).sum().reset_index()
# for practise

In [25]:
grpby_=[]
for i in range(tot_month):
    grpby_.append('df_{}'.format(i))
for i in range(tot_month):
    grpby_[i] = names_[i].drop(['date', 'item_price', 'date_block_num'],axis=1).groupby(['shop_id','item_id']).sum().reset_index()

In [26]:
for i in range(tot_month):
    grpby_[i]['month_num'] = i

In [27]:
grpby_[1]

,shop_id,item_id,item_cnt_day,month_num
0,0,30,31.0,1
1,0,31,11.0,1
2,0,32,10.0,1
3,0,33,3.0,1
4,0,35,14.0,1
...,...,...,...,...
59930,59,22020,2.0,1
59931,59,22024,1.0,1
59932,59,22087,1.0,1
59933,59,22130,2.0,1


In [28]:
x0 = grpby_[0].copy()
x1 = grpby_[1].copy()

In [29]:
x0 = x0.iloc[:20].drop('month_num',axis=1)
x1 = x1.iloc[:20].drop('month_num',axis=1)

# Merging every month including every single combination of shop and item

In [30]:
data = pd.merge(grpby_[0].drop('month_num',axis=1), grpby_[1].drop('month_num',axis=1),  how='outer', left_on=['shop_id','item_id'], right_on = ['shop_id','item_id'])

In [31]:
for i in range(2,tot_month):
    data = pd.merge(data, grpby_[i].drop('month_num',axis=1),  how='outer', left_on=['shop_id','item_id'], right_on = ['shop_id','item_id'])

In [32]:
data

,shop_id,item_id,item_cnt_day_x,item_cnt_day_y,item_cnt_day_x,item_cnt_day_y,item_cnt_day_x,item_cnt_day_y,item_cnt_day_x,item_cnt_day_y,...,item_cnt_day_x,item_cnt_day_y,item_cnt_day_x,item_cnt_day_y,item_cnt_day_x,item_cnt_day_y,item_cnt_day_x,item_cnt_day_y,item_cnt_day_x,item_cnt_day_y
0,0,32,6.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,33,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,35,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,43,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,51,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424119,59,20355,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
424120,59,20751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
424121,59,21361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
424122,59,21449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


In [33]:
col_names = []
for i in range(tot_month):
    col_names.append('month_{}'.format(i))

In [34]:
data.columns = list(data.columns[:2])+col_names

In [35]:
data = data.fillna(0)

In [36]:
data

,shop_id,item_id,month_0,month_1,month_2,month_3,month_4,month_5,month_6,month_7,...,month_24,month_25,month_26,month_27,month_28,month_29,month_30,month_31,month_32,month_33
0,0,32,6.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,33,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,35,1.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,43,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,51,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424119,59,20355,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
424120,59,20751,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
424121,59,21361,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
424122,59,21449,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


# Removing outliers

In [39]:
data.iloc[:,2:].describe()

,month_0,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,...,month_24,month_25,month_26,month_27,month_28,month_29,month_30,month_31,month_32,month_33
count,424124.000000,424124.000000,424124.000000,424124.000000,424124.000000,424124.000000,424124.000000,424124.000000,424124.000000,424124.000000,...,424124.000000,424124.000000,424124.000000,424124.000000,424124.000000,424124.000000,424124.000000,424124.000000,424124.000000,424124.000000
mean,0.310001,0.302011,0.346932,0.252733,0.252212,0.295623,0.275783,0.295411,0.314370,0.300716,...,0.261648,0.198522,0.193373,0.183501,0.170457,0.151168,0.148982,0.155801,0.171749,0.167536
std,1.511665,1.599241,2.051992,1.360014,2.010460,3.319192,2.737078,2.915463,3.957593,3.416915,...,3.762653,1.528047,1.492460,3.579918,2.840769,1.699408,1.577261,1.597581,4.919659,4.006454
min,-13.000000,-4.000000,-2.000000,-2.000000,-1.000000,-1.000000,-2.000000,-2.000000,-1.000000,-1.000000,...,-2.000000,-1.000000,-2.000000,-1.000000,-1.000000,-1.000000,-1.000000,-2.000000,-1.000000,-1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,169.000000,130.000000,259.000000,175.000000,504.000000,766.000000,799.000000,820.000000,950.000000,978.000000,...,1127.000000,300.000000,291.000000,1242.000000,742.000000,444.000000,482.000000,436.000000,1644.000000,2253.000000


In [48]:
data.iloc[:,2:].columns

Index(['month_0', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5',
       'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11',
       'month_12', 'month_13', 'month_14', 'month_15', 'month_16', 'month_17',
       'month_18', 'month_19', 'month_20', 'month_21', 'month_22', 'month_23',
       'month_24', 'month_25', 'month_26', 'month_27', 'month_28', 'month_29',
       'month_30', 'month_31', 'month_32', 'month_33'],
      dtype='object')

In [54]:
for i in data.iloc[:,2:].columns:
    neg_r=(data[i].mean() - 3*(data[i].std()))
    pos_r=(data[i].mean() + 3*(data[i].std()))
    data=(data[(data[i] > neg_r) & (data[i] < pos_r)])

In [55]:
data

,shop_id,item_id,month_0,month_1,month_2,month_3,month_4,month_5,month_6,month_7,...,month_24,month_25,month_26,month_27,month_28,month_29,month_30,month_31,month_32,month_33
1,0,33,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,43,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,51,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0,61,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0,75,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424119,59,20355,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
424120,59,20751,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
424121,59,21361,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
424122,59,21449,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


# import neccessary libraries

In [57]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [58]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

# Preparing data 

In [59]:
test

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [60]:
final_data = pd.merge(data, test.drop('ID',axis=1),  how='right', left_on=['shop_id','item_id'], right_on = ['shop_id','item_id']).fillna(0)

In [61]:
final_data=final_data.drop(['shop_id','item_id'], axis=1)

In [62]:
final_data

,month_0,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,...,month_24,month_25,month_26,month_27,month_28,month_29,month_30,month_31,month_32,month_33
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
214198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [110]:
# X_train = np.array(final_data.drop('month_33',axis=1)).reshape(-1,33,1)
# y_train = np.array(final_data['month_33']).reshape(-1,1)
# X_test = np.array(final_data.drop('month_0',axis=1)).reshape(-1,33,1)

In [111]:
# print(X_train.shape,y_train.shape,X_test.shape)

(214200, 33, 1) (214200, 1) (214200, 33, 1)


# Splitting Data

* Firstly we'll train our model on month0 - month32 and check the accuracy of our model on month33 
* Finally we'll fit the best model on month1-month33 to predict the sales for month34

In [63]:
X_train = np.array(final_data.drop('month_33',axis=1)).reshape(-1,33)
y_train = np.array(final_data['month_33']).reshape(-1,1)
X_test = np.array(final_data.drop('month_0',axis=1)).reshape(-1,33)

print(X_train.shape,y_train.shape,X_test.shape)

(214200, 33) (214200, 1) (214200, 33)


# Neural Network Model

In [64]:
model=Sequential()
model.add(Dense(150, activation='relu', input_shape=(X_train.shape[1],)))
for i in range(4):
    model.add(Dense(75, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [66]:
model.fit(X_train, y_train, epochs=50, batch_size=4096)

Epoch 1/50
214200/214200 [==============================] - 1s 6us/sample - loss: 0.1015
Epoch 2/50
214200/214200 [==============================] - 1s 6us/sample - loss: 0.1010
Epoch 3/50
214200/214200 [==============================] - 1s 6us/sample - loss: 0.1003
Epoch 4/50
214200/214200 [==============================] - 1s 7us/sample - loss: 0.0998
Epoch 5/50
214200/214200 [==============================] - 2s 9us/sample - loss: 0.0991
Epoch 6/50
214200/214200 [==============================] - 2s 9us/sample - loss: 0.0986
Epoch 7/50
214200/214200 [==============================] - 2s 9us/sample - loss: 0.0980
Epoch 8/50
214200/214200 [==============================] - 2s 9us/sample - loss: 0.0973
Epoch 9/50
214200/214200 [==============================] - 2s 9us/sample - loss: 0.0966
Epoch 10/50
214200/214200 [==============================] - 2s 9us/sample - loss: 0.0964
Epoch 11/50
214200/214200 [==============================] - 2s 9us/sample - loss: 0.0958
Epoch 12/50
214200/

In [67]:
ypred=model.predict(X_test)

In [68]:
ypred.shape

(214200, 1)

In [69]:
ypred

array([[0.23927192],
       [0.03288589],
       [0.10307579],
       ...,
       [0.04369986],
       [0.04369986],
       [0.04369986]], dtype=float32)

In [70]:
sample_submission['item_cnt_month'] = ypred

In [71]:
sample_submission.to_csv('submission_ann2.csv', index=False)